In [1]:
import pandas as pd
import psycopg2

# Database credentials
db_credentials = {
    "host": "branchhomeworkdb.cv8nj4hg6yra.ap-south-1.rds.amazonaws.com",
    "port": 5432,
    "user": "datascientist",
    "password": "47eyYBLT0laW5j9U24Uuy8gLcrN",
    "dbname": "branchdsprojectgps"
}

# Connect to the database
conn = psycopg2.connect(**db_credentials)

# Load data into DataFrames
loan_outcomes = pd.read_sql("SELECT * FROM loan_outcomes", conn)
gps_fixes = pd.read_sql("SELECT * FROM gps_fixes", conn)
user_attributes = pd.read_sql("SELECT * FROM user_attributes", conn)

# Close the connection
conn.close()

# Inspect data
print(loan_outcomes.head())
print(gps_fixes.head())
print(user_attributes.head())

C:\Users\91907\AppData\Local\Temp\ipykernel_18892\4013711881.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  loan_outcomes = pd.read_sql("SELECT * FROM loan_outcomes", conn)
C:\Users\91907\AppData\Local\Temp\ipykernel_18892\4013711881.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gps_fixes = pd.read_sql("SELECT * FROM gps_fixes", conn)


   user_id             application_at loan_outcome
0        1 2017-08-14 09:08:50.000000    defaulted
1        2 2016-05-17 10:10:12.447976       repaid
2        3 2016-10-20 10:07:20.459081    defaulted
3        4 2017-01-13 13:03:34.000000    defaulted
4        5 2016-11-03 15:41:39.124610       repaid
           gps_fix_at    server_upload_at  longitude  latitude  accuracy  \
0 2017-06-22 09:37:20 2017-06-22 09:43:42  36.840540 -1.294342      68.4   
1 2017-08-14 07:50:27 2017-08-14 09:05:27  36.895270 -1.341928    1409.0   
2 2017-06-13 10:34:29 2017-06-13 10:54:48  36.811903 -1.307220      68.4   
3 2017-06-18 12:16:20 2017-06-18 12:16:24  36.907049 -1.309984    1581.0   
4 2017-06-28 09:39:08 2017-06-28 09:58:12  36.839396 -1.280310    1396.0   

   altitude  bearing location_provider  user_id  
0       0.0      0.0             fused        1  
1       0.0      0.0             fused        1  
2       0.0      0.0             fused        1  
3       0.0      0.0             fuse

C:\Users\91907\AppData\Local\Temp\ipykernel_18892\4013711881.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user_attributes = pd.read_sql("SELECT * FROM user_attributes", conn)


In [2]:
merged_df = pd.merge(loan_outcomes, gps_fixes, on="user_id", how="outer")
final_df = pd.merge(merged_df, user_attributes, on="user_id", how="outer")

In [3]:
from sklearn.preprocessing import LabelEncoder

le_loan_outcome = LabelEncoder()
le_location_provider = LabelEncoder()

# Apply LabelEncoder to 'loan_outcome' and 'location_provider'
final_df["loan_outcome"] = le_loan_outcome.fit_transform(final_df["loan_outcome"])
final_df["location_provider"] = le_location_provider.fit_transform(final_df["location_provider"])

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,HistGradientBoostingClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, roc_curve
)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [5]:
def engineer_datetime_features(df):
    """
    Create meaningful features from datetime columns
    """
    # Convert datetime columns if they aren't already
    datetime_cols = ['application_at', 'gps_fix_at', 'server_upload_at']
    for col in datetime_cols:
        df[col] = pd.to_datetime(df[col])
    
    # Time differences
    df['time_since_gps_fix'] = (df['application_at'] - df['gps_fix_at']).dt.total_seconds() / 3600  # hours
    df['upload_delay'] = (df['server_upload_at'] - df['gps_fix_at']).dt.total_seconds() / 60  # minutes
    
    # Extract time-based features
    df['application_hour'] = df['application_at'].dt.hour
    df['application_day'] = df['application_at'].dt.day
    df['application_month'] = df['application_at'].dt.month
    df['is_weekend'] = df['application_at'].dt.dayofweek.isin([5, 6]).astype(int)
    
    # Drop original datetime columns
    df = df.drop(columns=datetime_cols)
    
    return df

# Prepare features
features_processed = final_df
features_processed = engineer_datetime_features(features_processed)

# Select final features for model
final_features = [
    'user_id', 'age', 'cash_incoming_30days',
    'time_since_gps_fix', 'upload_delay',
    'application_hour', 'application_day', 'application_month',
    'is_weekend'
]
target = final_df['loan_outcome']
X = features_processed[final_features]
y = target

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

import joblib

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(scaler, 'scaler.pkl')

# Model Training
model = HistGradientBoostingClassifier(
    random_state=42
)
model.fit(X_train_scaled, y_train)

# Predictions
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# Calculate metrics
metrics = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'F1': f1_score(y_test, y_pred),
    'ROC AUC': roc_auc_score(y_test, y_pred_proba)
}

print("\nModel Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")


Model Metrics:
Accuracy: 0.9991
Precision: 0.9996
Recall: 0.9991
F1: 0.9993
ROC AUC: 1.0000


In [6]:
import pickle

# Save the model
with open('loan_outcome_model.pkl', 'wb') as file:
    pickle.dump(model, file)
print("Model saved as 'loan_outcome_model.pkl'.")

# Load the model
with open('loan_outcome_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
print("Model loaded successfully.")

Model saved as 'loan_outcome_model.pkl'.
Model loaded successfully.
